### **Comparison of Random Number Generators**

In [1]:
"""
1. Consider a six months European put option with strike 55 on a non–dividend–paying
underlying asset with spot price 50 following a lognormal distribution with volatility 30%.
Assume that the risk-free rate is constant at 4%.
Compute the Black–Scholes value V of the option.
"""
import numpy as np
from scipy.stats import norm
import pandas as pd

# option parameters
K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

def bs_put(S0, K, T, r, sigma):
    """
    Black-Scholes formula for European put option
    """
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * np.exp(-q * T) * norm.cdf(-d1)

print("Black-Scholes value of the option: ", bs_put(S0, K, T, r, sigma))


Black-Scholes value of the option:  6.6166546586411705


**Inverse transform**

In [7]:
"""
Find an approximate option values using Monte Carlo simulations.

Let
Si = S(0)*exp((r-q-1/2*sigma^2)*T + sigma*sqrt(T)*zi), i = 1 : M,
Vi = e^(-rT)*max(K-Si, 0), i = 1 : M,
where zi, i = 1 : M, are independent samples of the standard normal variable obtained from N uniform samples.
Note that number of samples M = N_{A-R} and M = N_{B-M} corresponding to the Acceptance–
Rejection method and to the Marsaglia–Bray version of the Box–Muller method are smaller
than N. Report the approximate values \hat{V}(n) = \sum_{i=1}^n V_i/n and the corresponding approximation errors
|VBS - Vb(n)| in the tables below.

"""

# inverse transform
def inverse_transform(N, T, r, S0, sigma, K, q):
    """
    Inverse transform method
    """
    np.random.seed(42)
    U = np.random.uniform(0, 1, N)
    Z = norm.ppf(U)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)

    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(inverse_transform(n, T, r, S0, sigma, K, q))

res.to_csv('inverse_transform.csv', index=False)
res

,N,V,error
0,10000.0,6.730943,0.114289
1,20000.0,6.630084,0.013429
2,40000.0,6.621701,0.005046
3,80000.0,6.608744,0.007911
4,160000.0,6.608961,0.007694
5,320000.0,6.608559,0.008096
6,640000.0,6.612866,0.003789
7,1280000.0,6.612947,0.003707
8,2560000.0,6.617005,0.000351
9,5120000.0,6.616569,0.000086


**Acceptance Rejection Method**

In [10]:
%%time

# acceptance-rejection
def acceptance_rejection(N, T, r, S0, sigma, K, q):
    """
    Acceptance-rejection method
    """
    np.random.seed(42)
    Z = []
    for _ in range(N):
        u1, u2, u3 = np.random.rand(3)
        X = -np.log(u1)
        if u2 > np.exp(-(X-1)**2):
            continue
        else:
            if u3 <= 0.5:
                X = -X
            Z.append(X)
    Z = np.array(Z)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return len(P), P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())   

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'real_N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(acceptance_rejection(n, T, r, S0, sigma, K, q))
res.to_csv('acceptance_rejection.csv', index=False)
res

Wall time: 50.8 s


,N,real_N,V,error
0,10000.0,6402.0,6.527939,0.088716
1,20000.0,12839.0,6.504321,0.112334
2,40000.0,25508.0,6.499196,0.117459
3,80000.0,50855.0,6.528401,0.088254
4,160000.0,101767.0,6.525654,0.091000
5,320000.0,203447.0,6.538372,0.078282
6,640000.0,407607.0,6.549377,0.067278
7,1280000.0,814454.0,6.537215,0.079440
8,2560000.0,1629366.0,6.532508,0.084147
9,5120000.0,3258852.0,6.535198,0.081457


**Box-Muller Method**

In [16]:
%%time

# Marsaglia-Bray
def marsaglia_bray(N, T, r, s0, sigma, K, q):
    """
    Marsaglia-Bray method
    """
    np.random.seed(42)
    Z = []
    X = 2
    U = np.random.uniform(0, 1, N)
    for i in range(1, N//2+1):
        u1, u2 = U[2*i-2], U[2*i-1]
        u1 = 2 * u1 - 1
        u2 = 2 * u2 - 1
        X = u1**2 + u2**2
        if X > 1:
            continue
        Y = np.sqrt(-2 * np.log(X) / X)
        Z1 = u1 * Y
        Z2 = u2 * Y
        Z.extend([Z1, Z2])
    Z = np.array(Z)
    V = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    
    # calculate payoff
    P = np.exp(-r*T) * (K-V)
    P[P < 0] = 0

    return len(P), P.mean(), np.abs(bs_put(S0, K, T, r, sigma) - P.mean())   

K = 55
S0 = 50
sigma = 0.3
r = 0.04
T = 0.5
q = 0

nums = [10000, 20000, 40000, 80000, 160000, 320000, 640000, 1280000, 2560000, 5120000]
res = pd.DataFrame(columns=['N', 'real_N', 'V', 'error'])
for n in nums:
    res.loc[len(res)] = [n] + list(marsaglia_bray(n, T, r, S0, sigma, K, q))
res.to_csv('marsaglia_bray.csv', index=False)
res

Wall time: 16.2 s


,N,real_N,V,error
0,10000.0,7900.0,6.673365,0.056711
1,20000.0,15772.0,6.613423,0.003232
2,40000.0,31546.0,6.615431,0.001224
3,80000.0,63030.0,6.615819,0.000836
4,160000.0,125844.0,6.605458,0.011197
5,320000.0,251454.0,6.615751,0.000904
6,640000.0,503010.0,6.627872,0.011217
7,1280000.0,1005484.0,6.626227,0.009572
8,2560000.0,2010400.0,6.624078,0.007424
9,5120000.0,4020898.0,6.618746,0.002091
